# Libraries

In [7]:
import pandas as pd
import numpy as np
from binance.client import Client
import websocket
import json
import threading
import time
from datetime import datetime

# 1. Extract

Websocket

In [27]:
# Shared state
price_websocket = None

def on_message(ws, message):
    global price_websocket
    data = json.loads(message)
    wdata = data['k']
    price = float(wdata['c'])
    price_websocket = price

def on_open(ws):
    payload = {"method": "SUBSCRIBE",
            "params": ["btcusdt@kline_1s"],
               "id" = 1}
    ws.send(json.dumps(payload))
def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status, close_message):
    print("WebSocket closed")

def get_websocket_data():
    ws = websocket.WebSocketApp("wss://stream.binance.com:9443/ws",
                                on_open=on_open,
                                on_message=on_message,
                                on_error=on_error,
                                on_close=on_close )
    ws.run_forever()


ws_thread = threading.Thread(target=get_websocket_data, daemon = True)
ws_thread.start()

main_df = pd.DataFrame(columns=["timestamp", "price_websocket"])
for i in range(100):
    timestamp = datetime.utcnow().replace(microsecond=0)
    main_df = pd.concat([main_df, pd.DataFrame([{"timestamp": timestamp, "price_websocket": price_websocket}])], ignore_index=True)
    time.sleep(1)


main_df

{'e': 'kline', 'E': 1746207194040, 's': 'BTCUSDT', 'k': {'t': 1746207193000, 'T': 1746207193999, 's': 'BTCUSDT', 'i': '1s', 'f': 4870451053, 'L': 4870451054, 'o': '97325.99000000', 'c': '97326.00000000', 'h': '97326.00000000', 'l': '97325.99000000', 'v': '0.00533000', 'n': 2, 'x': True, 'q': '518.74752870', 'V': '0.00020000', 'Q': '19.46520000', 'B': '0'}}
{'e': 'kline', 'E': 1746207195015, 's': 'BTCUSDT', 'k': {'t': 1746207194000, 'T': 1746207194999, 's': 'BTCUSDT', 'i': '1s', 'f': 4870451055, 'L': 4870451055, 'o': '97326.00000000', 'c': '97326.00000000', 'h': '97326.00000000', 'l': '97326.00000000', 'v': '0.00015000', 'n': 1, 'x': True, 'q': '14.59890000', 'V': '0.00015000', 'Q': '14.59890000', 'B': '0'}}
{'result': None, 'id': None}
{'e': 'kline', 'E': 1746207196020, 's': 'BTCUSDT', 'k': {'t': 1746207195000, 'T': 1746207195999, 's': 'BTCUSDT', 'i': '1s', 'f': 4870451056, 'L': 4870451066, 'o': '97326.00000000', 'c': '97326.00000000', 'h': '97326.00000000', 'l': '97326.00000000', 'v':

KeyboardInterrupt: 

API

In [18]:
api_key = "S0su5HSh9NeSfo9hD0r8LWSoxqX2LvgNa2QmlonmkQblmOonsktOogczff6SDi01"
api_secret = "bWrYTb4bVYIA8AVSTjyeZzOOzRMjnX2L9c5PwrHD3am1639QMpLBk9klU9pULA6x"

client = Client(api_key, api_secret)

In [19]:
def get_historical_klines(symbol, interval, lookback):
    try:
        klines = client.get_historical_klines(symbol, interval, lookback)
        raw_df = pd.DataFrame(klines, columns = [
            "timestamp", "open", "high", "low", "close", "volume",
            "close_time", "quote_asset_volume", "number_of_trades", "taker_buy_base_asset_volume",
            "taker_buy_quote_asset_volume", "ignore"
        ])
        raw_df = raw_df[["timestamp", "close"]]
        raw_df["timestamp"] = pd.to_datetime(raw_df["timestamp"], unit = "ms")
        raw_df.set_index("timestamp", inplace = True)
        raw_df = raw_df.astype(float)
        return raw_df
    except Exception as e:
        raise Exception(f"Error")

In [20]:
symbol = "BTCUSDT"
interval = "1s"
lookback = "100 sec ago UTC"

raw_df = get_historical_klines(symbol, interval, lookback)
raw_df.columns = ['price_rest']
raw_df

,price_rest
timestamp,
2025-05-02 16:25:41,97733.77
2025-05-02 16:25:42,97733.77
2025-05-02 16:25:43,97733.76
2025-05-02 16:25:44,97733.77
2025-05-02 16:25:45,97733.76
...,...
2025-05-02 16:27:16,97734.08
2025-05-02 16:27:17,97730.57
2025-05-02 16:27:18,97730.57


In [21]:
btc_df = raw_df.merge(main_df, on='timestamp')
btc_df

,timestamp,price_rest,price_webstocket
0,2025-05-02 16:25:41,97733.77,97733.76
1,2025-05-02 16:25:42,97733.77,97733.76
2,2025-05-02 16:25:43,97733.76,97733.77
3,2025-05-02 16:25:44,97733.77,97733.76
4,2025-05-02 16:25:45,97733.76,97733.77
...,...,...,...
81,2025-05-02 16:27:02,97782.98,97778.31
82,2025-05-02 16:27:03,97782.98,97782.98
83,2025-05-02 16:27:04,97782.98,97782.98
84,2025-05-02 16:27:05,97782.98,97782.98
